In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install langchain-core langchain-openai pydantic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 32.3 MB/s eta 0:00:00


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from sklearn.metrics import classification_report
import time
import os
from tqdm import tqdm

In [ ]:
path = '/content/drive/MyDrive/MIAX/TRABAJO FINAL DE MASTER DEFINITIVO/news_second_classified.csv'
df_trainning = pd.read_csv(path)
df_trainning

<ipython-input-5-8ed57ea5a637>:2: DtypeWarning: Columns (5,8,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df_trainning = pd.read_csv(path)


,id,Publication Date,title,content,source,sentiment_body,sentiment_body_score,company,sector,url,ticker,description,text,esg_pred,esg_type
0,3.0,2024-07-18 14:19:34+00:00,How ToxMod's AI impacted toxicity in Call of D...,It's no secret Call of Duty has toxic players....,DNyuz,neutral,0.48,activision,Communication,NaN,ATVI,NaN,How ToxMod's AI impacted toxicity in Call of D...,1,1
1,4.0,2024-07-18 14:04:50+00:00,How ToxMod's AI impacted toxicity in Call of D...,It's no secret Call of Duty has toxic players....,Allusanewshub,neutral,0.54,activision,Communication,NaN,ATVI,NaN,How ToxMod's AI impacted toxicity in Call of D...,1,1
2,5.0,2024-05-27 07:03:20+00:00,"Meta, Activision Sued by Families of Uvalde Sc...",In the wake of the tragic shooting at Robb Ele...,Tech Times,neutral,0.92,activision,Communication,NaN,ATVI,NaN,"Meta, Activision Sued by Families of Uvalde Sc...",1,1
3,8.0,2024-04-02 15:34:29+00:00,Federal racial discrimination lawsuit against ...,"'In Tesla's narrative, the agencies learned no...",Head Topics,negative,0.41,activision,Communication,NaN,ATVI,NaN,Federal racial discrimination lawsuit against ...,1,1
4,11.0,2024-01-24 03:28:56+00:00,Activision Will Pay $50 Million to Settle Work...,Activision Blizzard will pay roughly $50 milli...,Mytimesnow,positive,0.64,activision,Communication,NaN,ATVI,NaN,Activision Will Pay $50 Million to Settle Work...,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431215,NaN,2025-03-21 10:50:51,Nike is starting to show green-shoots with inn...,"Andrea Andreeva, Piper Sandler analyst, joins ...",CNBC Television,NaN,NaN,nike,NaN,https://www.youtube.com/watch?v=yrwUTpiLvsI,NKE,NaN,Nike is starting to show green-shoots with inn...,1,2
431216,NaN,2025-03-21 11:19:37,"Nike Stock Plummets on Dismal Forecast, Bear N...",Brushing off a fiscal third-quarter earnings a...,Schaeffers Research,NaN,NaN,nike,NaN,https://www.schaeffersresearch.com/content/new...,NKE,NaN,"Nike Stock Plummets on Dismal Forecast, Bear N...",1,2
431217,NaN,2025-03-21 11:20:15,Nike Turnaround Hits Snags Amid Inventory Rese...,Nike's turnaround effort is facing challenges ...,Bloomberg Markets and Finance,NaN,NaN,nike,NaN,https://www.youtube.com/watch?v=T4MRIaMz2ks,NKE,NaN,Nike Turnaround Hits Snags Amid Inventory Rese...,1,2
431218,NaN,2025-03-21 12:31:54,Why Nike Stock Got Tripped Up Today,Shares of Nike (NKE -5.16%) fell Friday mornin...,The Motley Fool,NaN,NaN,nike,NaN,https://www.fool.com/investing/2025/03/21/why-...,NKE,NaN,Why Nike Stock Got Tripped Up Today. Shares of...,1,2


In [ ]:
df_trainning['esg_type'].value_counts()

,count
esg_type,
2,169286
1,151232
0,110702


In [ ]:
os.environ["OPENAI_API_KEY"] = 'xxxxxx'

In [ ]:
news_sample = df_trainning.sample(n=8000, random_state=33).reset_index(drop=True)

In [ ]:
class Classification(BaseModel):
    sentiment: str = Field(..., description="Text Sentiment", enum=["bullish", "bearish", "neutral"])

In [ ]:
tagging_prompt = ChatPromptTemplate.from_template(
    """
You are an expert financial analyst specialized in ESG-related news.

Analyze the following news article and classify its overall sentiment towards the company's performance as one of:
- bullish (positive impact expected)
- bearish (negative impact expected)
- neutral (no significant impact)

Only respond with one of these three words: bullish, bearish, or neutral.

Text:
{input}
"""
)

In [ ]:
model = ChatOpenAI(model="gpt-4o", temperature=0).with_structured_output(Classification)
tagging_chain = tagging_prompt | model

In [ ]:
sentiments = []

start = time.time()

for i, row in tqdm(news_sample.iterrows(), total=len(news_sample)):
    text = row['text']
    try:
        result = tagging_chain.invoke({"input": text})
        sentiments.append(result.sentiment)
    except Exception as e:
        sentiments.append(None)

print("Tiempo total:", round(time.time() - start, 2), "segundos")
news_sample['predicted_sentiment'] = sentiments

100%|██████████| 8000/8000 [1:24:03<00:00,  1.59it/s]

⏱️ Tiempo total: 5043.3 segundos


In [ ]:
news_sample['predicted_sentiment'].value_counts()

,count
predicted_sentiment,
neutral,3033
bearish,2655
bullish,2312


In [ ]:
news_sample.to_csv('/content/drive/MyDrive/MIAX/TRABAJO FINAL DE MASTER DEFINITIVO/sentiment_sample_e_s_g.csv', index=False)